In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC

In [36]:
def ml_magic(df, type_of_alg, looking_values, test_size = 0.2, minmax = True):


    X_train, X_test, y_train, y_test = train_test_split(df.drop(['centre_metal_Ni_=1 '],axis =1),df['centre_metal_Ni_=1 '],test_size=test_size)

    if type_of_alg == 'ada':
        model = AdaBoostClassifier(n_estimators=200)
    elif type_of_alg == 'knn':
        model = KNeighborsClassifier(n_neighbors=9)
    elif type_of_alg == 'r_forest':
        model = RandomForestClassifier(n_estimators=300)
    elif type_of_alg == 'b_forest':
        model = GradientBoostingClassifier(n_estimators=300, learning_rate=0.2, max_depth=1, random_state=1)
    elif type_of_alg == 'svc':
        model = SVC(C=0.7,degree = 4)

    model.fit(X_train, y_train)

    m = confusion_matrix(y_test,model.predict(X_test))

    return((m[1][1]+m[0][0])/len(y_test))


df = pd.read_csv('population.csv')

df = df.drop(['Unnamed: 0', 'energia'], axis = 1)
df

models = ['ada','knn','r_forest','b_forest','svc']
estimators = [50,100,150,200,250,300]

best_pred = []
for model in models:
    for estimator in estimators:
        for _ in range(10):
            sub_pred = []
            sub_pred.append(model)
            sub_pred.append(estimator)
            sub_pred.append(ml_magic(df, type_of_alg = model, looking_values='energia_zredukowana'))
            best_pred.append(sub_pred)
best_predcd 


[['ada', 50, 0.9],
 ['ada', 50, 0.85],
 ['ada', 50, 0.9],
 ['ada', 50, 0.85],
 ['ada', 50, 0.95],
 ['ada', 50, 0.95],
 ['ada', 50, 0.85],
 ['ada', 50, 1.0],
 ['ada', 50, 0.95],
 ['ada', 50, 0.8],
 ['ada', 100, 0.95],
 ['ada', 100, 0.95],
 ['ada', 100, 0.95],
 ['ada', 100, 1.0],
 ['ada', 100, 0.85],
 ['ada', 100, 0.9],
 ['ada', 100, 0.9],
 ['ada', 100, 0.85],
 ['ada', 100, 0.95],
 ['ada', 100, 0.9],
 ['ada', 150, 1.0],
 ['ada', 150, 1.0],
 ['ada', 150, 0.8],
 ['ada', 150, 0.95],
 ['ada', 150, 0.75],
 ['ada', 150, 0.9],
 ['ada', 150, 0.9],
 ['ada', 150, 1.0],
 ['ada', 150, 0.9],
 ['ada', 150, 0.85],
 ['ada', 200, 1.0],
 ['ada', 200, 0.8],
 ['ada', 200, 0.9],
 ['ada', 200, 0.95],
 ['ada', 200, 0.7],
 ['ada', 200, 0.95],
 ['ada', 200, 0.8],
 ['ada', 200, 0.9],
 ['ada', 200, 0.95],
 ['ada', 200, 0.95],
 ['ada', 250, 1.0],
 ['ada', 250, 0.9],
 ['ada', 250, 0.95],
 ['ada', 250, 0.85],
 ['ada', 250, 0.9],
 ['ada', 250, 0.9],
 ['ada', 250, 0.85],
 ['ada', 250, 0.95],
 ['ada', 250, 0.95],
 ['ada

In [35]:
df.columns

Index(['liczba Ni', 'centre_metal_Ni_=1 ', 'energia_zredukowana',
       'ni_path_ncentre', 'ni_path_centre', 'cu_with_centre',
       'cu_path_ncentre', 'srodek_masy', '0%Ni', '8%Ni', '16%Ni', '25%Ni',
       '33%Ni', '41%Ni', '50%Ni', 'usieciowanie', 'usieciowanie_cu'],
      dtype='object')